In [4]:
print("Welcome")

Welcome


In [25]:
import requests
from bs4 import BeautifulSoup, SoupStrainer

import warnings
#warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

def get_list_of_questions_url(base_url, questions_link):
    #get page dom
    #http://docs.python-requests.org/en/master/
    response = requests.get(questions_link)
    
    #https://www.crummy.com/software/BeautifulSoup/bs4/doc/
    soup = BeautifulSoup(response.content, "lxml",  parse_only=SoupStrainer("a", href=True))

    list = []
    for link in soup('a'):
        actualTag = link['href']
        if("problems" not in actualTag):
            continue

        if(actualTag.startswith("/")):
            actualTag = base_url + actualTag

        if(actualTag.endswith("/")):
            actualTag = actualTag[:-1]
            
        #print("\n ",actualTag)
        list.append(actualTag)
    
    return list 
    
def aggregate_all_questions_url():      
    WEBSITE_BASE_URL = "https://www.codechef.com"
    QUESTIONS_LIST_URLS = ["https://www.codechef.com/problems/school",
                 "https://www.codechef.com/problems/easy", 
                 "https://www.codechef.com/problems/medium",
                 "https://www.codechef.com/problems/hard",
                 "https://www.codechef.com/problems/challenge",
                 "https://www.codechef.com/problems/extcontest"]
    
    SKIP_EXTRA_URLS = QUESTIONS_LIST_URLS[:]
    SKIP_EXTRA_URLS.append("http://www.codechef.com/problemsetting")
    
    global list_of_problems_url
    for url in QUESTIONS_LIST_URLS[:-5]:
        list_of_problems_url += get_list_of_questions_url(WEBSITE_BASE_URL, url)
      
    #removing extra links
    for question_url in SKIP_EXTRA_URLS:
        list_of_problems_url = [url for url in list_of_problems_url if url != question_url]
        
    print(len(list_of_problems_url))
    #print(list_of_problems_url)


In [32]:
list_of_problems_url = []
aggregate_all_questions_url()


163


In [51]:
import os

if not os.path.exists("tmp"):
    os.makedirs("tmp")
    
def save_to_file(path, content):
    file = open(path , 'w')
    file.write(content)
    file.close()
    print("File saved : ", path)

In [54]:
import time

#process each url to extract features
for url in list_of_problems_url[:2]:
    #Careful don't call it without sleep, DDOS
    time.sleep(.5)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "lxml")
    content = soup.find("div", {"id": "problem-page"})
    #print(content)
    filename = "tmp" + url[url.rfind("/"):] + ".txt"
    save_to_file(filename, content.text)
    

File saved :  tmp/H1.txt
File saved :  tmp/TRACE.txt
